In [2]:
from glob import glob
import os
import sys
from tqdm import tqdm

In [3]:
data_path = r'E:\m_0.65_0.85'
fileNames = glob(data_path+'\\*0.txt')
fileNames=fileNames

In [19]:
# dump
def dump(sr=None, size=512):
    
    M,ALPHA=[],[]
    if sr is None: # 从配置文件中获取 sr
        with open(data_path+'\\输入参数文本.txt') as _f:
            _paras = _f.readlines()
            sr = int(_paras[2].split('\t')[0]) + 1

    # --------------------------------------------------------------通用begin
    with open(fileNames[0]) as _f:
        simple_size = sys.getsizeof(_f.read())/1024/1024 # Mb

    size = size # 2048:2GB
    num = int(size/simple_size)+1 # 每一次存的大小是 num个读取的文件
    N = int(len(fileNames)//num)  # 存N次大小为num个的文件
    last_num = len(fileNames)%num # 最后一次存的文件是 last_num个读取的文件
    begin=0
    print('size,simple_size,num,N,last_num\n',size,simple_size,num,N,last_num)
    for c_i in tqdm(range(N+1)):
        if c_i==N:
            num = last_num
        last = begin+num
        filename = fileNames[begin:last]
    # --------------------------------------------------------------通用end
        i1 = np.zeros((len(filename), sr))
        i2 = np.zeros((len(filename), sr))
        dop = np.zeros((len(filename), sr))
        for idx, i in enumerate(filename):
            baseName = os.path.basename(i) 
            m,alpha = baseName.split(',')
            alpha = '.'.join(alpha.split('.')[:-1])
            m = m.split('=')[-1]
            alpha=alpha.split('=')[-1]
            M.append(m)
            ALPHA.append(alpha)

            i1[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).I1.to_numpy().astype(np.float32)
            i2[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).I2.to_numpy().astype(np.float32)
            dop[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).DOP.to_numpy().astype(np.float32)
    # --------------------------------------------------------------- 通用bedin
        out_path = r'E:\out'
        np.save(os.path.join(out_path, f'i1{c_i}.npy'),i1)
        np.save(os.path.join(out_path, f'i2{c_i}.npy'),i2)
        np.save(os.path.join(out_path, f'dop_{c_i}.npy'),dop)
        np.savez(os.path.join(out_path, f'M_ALPHA{c_i}.npz'),M=M,ALPHA=ALPHA)

        begin = last
    # --------------------------------------------------------------- 通用end
    return out_path


In [20]:
target_path = dump()

size,simple_size,num,N,last_num
 512 0.1297311782836914 3947 0 2000


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:03<00:00, 63.48s/it]


In [116]:
a=np.ones((10,5))
b=np.zeros((3,6))
np.savez('ab.npz',a=a,b=b)

In [117]:
res = np.load('ab.npz')

In [118]:
res['a']

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [96]:
# load
def load_data(path):
    fileNames = glob(os.path.join(path,'*'))
    
    name_i1 = [i for i in fileNames if 'i1' in i]
    name_i2 = [i for i in fileNames if 'i2' in i]
    name_dop = [i for i in fileNames if 'dop' in i]
    name_m_alpha = [i for i in fileNames if 'M_ALPHA' in i]
    L = len(name_dop) # 3种文件数目相同
    c = 0
    while True:
        i1 = np.load(name_i1[c%L])
        i2 = np.load(name_i2[c%L])
        dop = np.load(name_dop[c%L])
        res = np.load(name_m_alpha[c%L])
        M, ALPHA = res['M'], res['ALPHA']
        yield i1, i2, dop, M, ALPHA
        c += 1


In [97]:
data = load_data(target_path)

In [103]:
%%time
dop=next(data)
dop[0].shape

Wall time: 73.8 ms


(2000, 2048)